Nesta análise axploratória será respondida as sequintes perguntas:

- 1- Quais são as categorias mais Ccmuns no IMDB?
- 2- Qual a mediana de avaliação por gênero?
- 3- Qual a mediana de avaliação em relação ao ano de estréia?
- 4- Qual a relação entre duração e gênero?
- 5- Qual o número de filmes produzidos por país?
- 6- Quais são os Top 10 melhores filmes?
- 7- Quais são os Top 10 piores filmes?

In [ ]:
# Instalando o pacote imdb-sqlite
!pip install -q imdb-sqlite

In [ ]:
# Instalando o pacote pycountry para avalição da base de dados por países
!pip install -q pycountry

In [ ]:
# Importando os pacotes
import re
import time
import sqlite3
import pycountry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
sns.set_theme(style = "whitegrid")

In [ ]:
#Baixando a base de dados

!imdb-sqlite

In [ ]:
# Realizando a conexão no banco de dados

conn = sqlite3.connect("imdb.db")

In [ ]:
# Extraindo a lista de tabelas através de uma query SQL

tabelas = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type = 'table'", conn)

In [ ]:
# Convertendo o dataframe em uma lista

tabelas = tabelas["Table_Name"].values.tolist()

In [ ]:
# Analisando o esquema de cada tabela na base

for tabela in tabelas:
    consulta = "PRAGMA TABLE_INFO({})".format(tabela)
    resultado = pd.read_sql_query(consulta, conn)
    print("Tabela:", tabela)
    display(resultado)
    print("\n")

Visualizando os dados de cada tabela da base

In [ ]:
table1 = '''SELECT * FROM people'''
view1 = pd.read_sql_query(table1, conn)

display(view1)

In [ ]:
table2 = '''SELECT * FROM titles'''
view2 = pd.read_sql_query(table2, conn)

display(view2)

In [ ]:
table3 = '''SELECT * FROM akas'''
view3 = pd.read_sql_query(table3, conn)

display(view3)

In [ ]:
table4 = '''SELECT * FROM crew'''
view4 = pd.read_sql_query(table4, conn)

display(view4)

In [ ]:
table5 = '''SELECT * FROM episodes'''
view5 = pd.read_sql_query(table5, conn)

display(view5)

In [ ]:
table6 = '''SELECT * FROM ratings'''
view6 = pd.read_sql_query(table6, conn)

display(view6)

## 1- Quais São as Categorias Mais Comuns no IMDB?

In [ ]:
# Criando a query SQL

consulta1 = '''SELECT type, COUNT(*) AS COUNT FROM titles GROUP BY type''' 

In [ ]:
# armazenando o resultado da query na variável resultado1

resultado1 = pd.read_sql_query(consulta1, conn)

In [ ]:
display(resultado1)

In [ ]:
# Percentual para cada tipo

resultado1['percentual'] = (resultado1['COUNT'] / resultado1['COUNT'].sum()) * 100

In [ ]:
display(resultado1)

In [ ]:
#Para uma melhor visualização será analisado os 3 tipos mais comuns, e o restante será visto como "others"

# Criando um dicionário vazio
others = {}

# Filtrando o percentual em 5% e somando o total
others['COUNT'] = resultado1[resultado1['percentual'] < 5]['COUNT'].sum()

# Armazenando o percentual total
others['percentual'] = resultado1[resultado1['percentual'] < 5]['percentual'].sum()

# Nomeando o tipo
others['type'] = 'others'

In [ ]:
# Visualiza
others

In [ ]:
# Filtrando o dataframe de resultado
resultado1 = resultado1[resultado1['percentual'] > 5]

In [ ]:
# Fazendo um append com o dataframe de outras categorias
resultado1 = resultado1.append(others, ignore_index = True)

In [ ]:
# Ordenando o resultado
resultado1 = resultado1.sort_values(by = 'COUNT', ascending = False)

In [ ]:
resultado1.head()

## 2- Qual a Mediana de Avaliação Por Gênero?

In [ ]:
consulta2 = '''
            SELECT rating, genres FROM 
            ratings JOIN titles ON ratings.title_id = titles.title_id 
            WHERE premiered <= 2022 AND type = 'movie'
            ''' 

In [ ]:
resultado2 = pd.read_sql_query(consulta2, conn)

In [ ]:
display(resultado2)

In [ ]:
# O código dessa célula cria um vetor o qual percorre o texto na coluna "genres", retornando os gêneros únicos de filmes

def retorna_generos(df):
    df['genres'] = df['genres'].str.lower().values
    temp = df['genres'].dropna()
    vetor = CountVectorizer(token_pattern = '(?u)\\b[\\w-]+\\b', analyzer = 'word').fit(temp)
    generos_unicos =  vetor.get_feature_names()
    generos_unicos = [genre for genre in generos_unicos if len(genre) > 1]
    return generos_unicos

In [ ]:
generos_unicos = retorna_generos(resultado2)

In [ ]:
generos_unicos

In [ ]:
# Criando listas vazias
genero_counts = []
genero_ratings = []

In [ ]:
for item in generos_unicos:
    
    # Retorna a contagem de filmes por gênero
    consulta = 'SELECT COUNT(rating) FROM ratings JOIN titles ON ratings.title_id=titles.title_id WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\''
    resultado = pd.read_sql_query(consulta, conn)
    genero_counts.append(resultado.values[0][0])
  
     # Retorna a avaliação de filmes por gênero
    consulta = 'SELECT rating FROM ratings JOIN titles ON ratings.title_id=titles.title_id WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\''
    resultado = pd.read_sql_query(consulta, conn)
    genero_ratings.append(np.median(resultado['rating']))

In [ ]:
df_genero_ratings = pd.DataFrame()
df_genero_ratings['genres'] = generos_unicos
df_genero_ratings['count'] = genero_counts
df_genero_ratings['rating'] = genero_ratings

df_genero_ratings = df_genero_ratings.sort_values(by = 'rating', ascending = False)

In [ ]:
df_genero_ratings

## 3- Qual a Mediana de Avaliação Em Relação ao Ano de Estréia?

In [ ]:
consulta3 = '''
            SELECT rating AS Rating, premiered FROM 
            ratings JOIN titles ON ratings.title_id = titles.title_id 
            WHERE premiered <= 2022 AND type = 'movie'
            ORDER BY premiered
            ''' 

In [ ]:
resultado3 = pd.read_sql_query(consulta3, conn)

In [ ]:
display(resultado3)

In [ ]:
# Calculando a mediana
ratings = []
for year in set(resultado3['premiered']):
    ratings.append(np.median(resultado3[resultado3['premiered'] == year]['Rating']))

In [ ]:
anos = list(set(resultado3['premiered']))

In [ ]:
# Plot
plt.figure(figsize = (16,8))
plt.plot(anos, ratings)
plt.xlabel('\nAno')
plt.ylabel('Mediana de Avaliação')
plt.title('\nMediana de Avaliação Em Relação ao Ano de Estréia\n')
plt.show()

## 4- Qual a Relação Entre Duração e Gênero?

In [ ]:
consulta4 = '''
            SELECT AVG(runtime_minutes) Runtime, genres 
            FROM titles 
            WHERE type = 'movie'
            AND runtime_minutes != 'NaN'
            GROUP BY genres
            ''' 

In [ ]:
resultado4 = pd.read_sql_query(consulta4, conn)

In [ ]:
generos_unicos = retorna_generos(resultado4)

In [ ]:
generos_unicos

In [ ]:
# Calculando a duração por gênero

genero_runtime = []
for item in generos_unicos:
    consulta = 'SELECT runtime_minutes Runtime FROM  titles  WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\' AND Runtime!=\'NaN\''
    resultado = pd.read_sql_query(consulta, conn)
    genero_runtime.append(np.median(resultado['Runtime']))

In [ ]:
# Prepara o dataframe
df_genero_runtime = pd.DataFrame()
df_genero_runtime['genre'] = generos_unicos
df_genero_runtime['runtime'] = genero_runtime

In [ ]:
# Ordena os dados
df_genero_runtime = df_genero_runtime.sort_values(by = 'runtime', ascending = False)

In [ ]:
df_genero_runtime

## 5- Qual o Número de Filmes Produzidos Por País?

In [ ]:
consulta5 = '''
            SELECT region, COUNT(*) Number_of_movies FROM 
            akas JOIN titles ON 
            akas.title_id = titles.title_id
            WHERE region != 'None'
            AND type = \'movie\'
            GROUP BY region
            ''' 

In [ ]:
resultado5 = pd.read_sql_query(consulta5, conn)

In [ ]:
display(resultado5)

In [ ]:
nomes_paises = []
contagem = []

In [ ]:
# Loop para obter o país de acordo com a região
for i in range(resultado5.shape[0]):
    try:
        coun = resultado5['region'].values[i]
        nomes_paises.append(pycountry.countries.get(alpha_2 = coun).name)
        contagem.append(resultado5['Number_of_movies'].values[i])
    except: 
        continue

In [ ]:
df_filmes_paises = pd.DataFrame()
df_filmes_paises['country'] = nomes_paises
df_filmes_paises['Movie_Count'] = contagem

In [ ]:
df_filmes_paises = df_filmes_paises.sort_values(by = 'Movie_Count', ascending = False)

In [ ]:
df_filmes_paises.head(20)

## 6- Quais São os Top 10 Melhores Filmes?

In [ ]:
consulta6 = '''
            SELECT primary_title AS Movie_Name, genres, rating
            FROM 
            titles JOIN ratings
            ON  titles.title_id = ratings.title_id 
            WHERE titles.type = 'movie' AND ratings.votes >= 25000
            ORDER BY rating DESC
            LIMIT 10          
            ''' 

In [ ]:
top10_melhores_filmes = pd.read_sql_query(consulta6, conn)

In [ ]:
display(top10_melhores_filmes)

## 7- Quais São os Top 10 Piores Filmes?

In [ ]:
consulta7 = '''
            SELECT primary_title AS Movie_Name, genres, rating
            FROM 
            titles JOIN ratings
            ON  titles.title_id = ratings.title_id
            WHERE titles.type = 'movie' AND ratings.votes >= 25000
            ORDER BY rating ASC
            LIMIT 10
            ''' 

In [ ]:
top10_piores_filmes = pd.read_sql_query(consulta7, conn)

In [ ]:
display(top10_piores_filmes)